In [2]:
import cupy as cp
import numpy as np
import copy
import time
import matplotlib.pyplot as plt
import open3d as o3d
pcd_14 = o3d.io.read_point_cloud("/home/turin/Desktop/lizard_dataset_curated/2014/pcd14.pcd")
pcd_15 = o3d.io.read_point_cloud("/home/turin/Desktop/lizard_dataset_curated/2015/pcd15.pcd")
src_cp = copy.deepcopy(pcd_15)
trg_cp = copy.deepcopy(pcd_14)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 3
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

def prepare_dataset(source, target, voxel_size):
    print(":: Load two point clouds with initial pose.")
    # trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0],
    #                          [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    # source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: Apply fast global registration with distance threshold %.3f" \
            % distance_threshold)
    result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.pipelines.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

#FAST GLOBAL TRANSFORMATION
start = time.time()
voxel_size = 0.1  # 0.05 means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(src_cp, trg_cp,
    voxel_size)

result_fast = execute_fast_global_registration(source_down, target_down,
                                               source_fpfh, target_fpfh,
                                               voxel_size)
print("Fast global registration took %.3f sec.\n" % (time.time() - start))
print(result_fast)
src_cp.transform(result_fast.transformation)
draw_registration_result(pcd_15, pcd_14, result_fast.transformation)

:: Load two point clouds with initial pose.
:: Downsample with a voxel size 0.100.
:: Estimate normal with search radius 0.200.
:: Compute FPFH feature with search radius 0.300.
:: Downsample with a voxel size 0.100.
:: Estimate normal with search radius 0.200.
:: Compute FPFH feature with search radius 0.300.
:: Apply fast global registration with distance threshold 0.150
Fast global registration took 63.749 sec.

RegistrationResult with fitness=6.072631e-01, inlier_rmse=8.447069e-02, and correspondence_set size of 35183
Access transformation to get result.


In [4]:
src_col = copy.deepcopy(src_cp)
dist_error = src_cp.compute_point_cloud_distance(trg_cp)
# error = []
# for x in range(len(dist_error)):
#     error.append(dist_error[x])
error = np.asarray(dist_error)

In [26]:
e = error
mask = e<0.05
e[mask] = 0
np.count_nonzero(e), e.min(), e.max()

(1531838, 0.0, 1.7892995304529669)

In [27]:
def rgb(minimum, maximum, value):
    #minimum, maximum = float(minimum), float(maximum)
    ratio = (value-minimum) / (maximum - minimum)
    print(ratio.shape)
    r = 1-np.int32(1*ratio)
    b = 0+r*0.000001
    mask = e==0
    g = e*0
    g[mask] = .9
    return r, g, b
r,g,b = rgb(error.min(), e.max(), e)

(42312642,)


In [29]:
col = np.asarray([r,g,b], dtype=np.float32)
col = col.T
src_col.colors = o3d.pybind.utility.Vector3dVector(col)
o3d.visualization.draw_geometries([src_col, pcd_14])

In [38]:
o3d.visualization.draw_geometries([src_col])